# This notebook serves as walkthrough for planning an experiment for creation through the OT2.
### The following modules are used and should be in the same directory as this notebook: 
* **CreateSamples** is responsible for calculating sample information, which includes component weight fractions and stock volumes
* **OT2Commands** is responsible for setting up information to be interpretted and executed by opentrons **BASED ON THE LATEST API 2.3 and above**
* **OT2Graphing** contains graphing tools to help visualize and explore parameter spaces.

In [1]:
from Plan import CreateSamples
from Prepare import OT2Commands as ALH
from opentrons import simulate, execute, protocol_api

In [2]:
import pandas as pd
import time 
import numpy as np

In [3]:
def get_experimental_subset( QAS1, QAS2, volume_dataframe):
    
    if QAS1.endswith('-stock'):
        pass
    else:
        QAS1 = QAS1 +'-stock'
    
    if QAS1 == QAS2:
        subset_df = volume_dataframe[(volume_dataframe[QAS1] != 0)]
        subset_df=subset_df.loc[:,(subset_df !=0).any(axis=0)]
  
    else:
        if QAS2.endswith('-stock'):
            pass
        else:
            QAS2 = QAS2 +'-stock'  
        
        subset_df = volume_dataframe[(volume_dataframe[QAS1] != 0) | (volume_dataframe[QAS2]!= 0)]
        subset_df=subset_df.loc[:,(subset_df !=0).any(axis=0)]
    
    return subset_df
    

____
____

## Experimental Section

Define two dictionaries to keep track of the species involved in this experiment. 
For the DES Campaign we have 5 Quaternary Ammonium Salts (QAS) and 10 Hydrogen Bond Donors (HBD). 


In [4]:
QAS_dict= {'A0' : 'Choline Chloride',
           'A1' : 'Acetylcholine Chloride',
           'A2' : 'Tetraethylammonium Chloride',
           'A3' : 'Tetrapropylammonium Bromide',
           'A4' : 'Tetraethylammonium Iodide'}
HBD_dict= {'A1': 'Ethylene Glycol',
           'A2':'Glycerol',
           'A3' : 'Acetamide',
           'A4': 'N,N-dimethylurea',
           'A5': '3-phenylpropionic acid',
           'A6': 'Urea',
           'A7': 'L-serine',
           'A8': '4-amino triazole',
           'A9': 'Xylitol',
           'A10': 'Phenylacetic acid'}

There is a total of 50 different combinations of species. 

We plan on formulating 12 different mole fractions between each combination of QAS and HBD. 

The required volumes of each components, based on their stock solutions concentration and the total sampple volume (600uL), was calcuated using the ` Calcuate_Sample_Volume` Jupyter Notebook which can also be found in the Supplementary Material

The code below uses the functions of the OT2-DOE GitHub [repo](https://github.com/pozzo-research-group/OT2-DOE) to run the campaign. 

In [5]:
volume_data = pd.read_csv('./SM_final_volume_table.csv', index_col=0)

The total stock volumes required for the entire campaign is as follows: 

`Note: the volume units displayed are mL`

In [11]:
stock_volumes = ALH.calculate_total_volumes(volume_data)
stock_volumes

Choline Chloride-stock               36.881370
Acetylcholine Chloride-stock         36.881370
Tetraethylammonium Chloride-stock    40.840633
Tetrapropylammonium Bromide-stock    46.241007
Tetraethylammonium Iodide-stock      54.464040
Ethylene Glycol-stock                11.591015
Glycerol-stock                       11.591015
Acetamide-stock                      11.591015
N,N-dimethylurea-stock               11.591015
3-phenylpropionic acid-stock         17.185167
Urea-stock                           11.591015
L-serine-stock                       19.956755
4-amino triazole-stock               15.224249
Xylitol-stock                        17.185167
Phenylacetic acid-stock              17.185167
dtype: float64

Since the robot has a limited space for labware, stock reservoirs and pipette racks, _the campaign will have to be split over different runs_. 

    **NOTE:** 
    The first run will only include Choline Chloride as the QAS with all its coombinations with the 10 HBD species
    Each successive run will involve 2 different QAS and all their combinations with the 10 HBD species

In [14]:
run0= get_experimental_subset(QAS_dict['A0'], QAS_dict['A0'], volume_data)
stock_volumes_run0 = ALH.calculate_total_volumes(run0)
stock_volumes_run0

Choline Chloride-stock          36.881370
Ethylene Glycol-stock            2.898840
Glycerol-stock                   2.898840
Acetamide-stock                  2.898840
N,N-dimethylurea-stock           2.898840
3-phenylpropionic acid-stock     4.096246
Urea-stock                       2.898840
L-serine-stock                   4.645691
4-amino triazole-stock           3.690000
Xylitol-stock                    4.096246
Phenylacetic acid-stock          4.096246
dtype: float64

In [15]:
run1= get_experimental_subset(QAS_dict['A1'], QAS_dict['A2'], volume_data)
stock_volumes_run1 =  ALH.calculate_total_volumes(run1)
stock_volumes_run1

Acetylcholine Chloride-stock         36.881370
Tetraethylammonium Chloride-stock    40.840633
Ethylene Glycol-stock                 5.408743
Glycerol-stock                        5.408743
Acetamide-stock                       5.408743
N,N-dimethylurea-stock                5.408743
3-phenylpropionic acid-stock          7.786246
Urea-stock                            5.408743
L-serine-stock                        8.905139
4-amino triazole-stock                6.970405
Xylitol-stock                         7.786246
Phenylacetic acid-stock               7.786246
dtype: float64

In [16]:
run2= get_experimental_subset(QAS_dict['A3'], QAS_dict['A4'], volume_data)
stock_volumes_run2 =  ALH.calculate_total_volumes(run2)
stock_volumes_run2

Tetrapropylammonium Bromide-stock    46.241007
Tetraethylammonium Iodide-stock      54.464040
Ethylene Glycol-stock                 3.283432
Glycerol-stock                        3.283432
Acetamide-stock                       3.283432
N,N-dimethylurea-stock                3.283432
3-phenylpropionic acid-stock          5.302674
Urea-stock                            3.283432
L-serine-stock                        6.405925
4-amino triazole-stock                4.563844
Xylitol-stock                         5.302674
Phenylacetic acid-stock               5.302674
dtype: float64

### Check for Pipette tips to use

**Each of the QAS selected** per run will be distributed using **a single pipette**, as there is no other sample in the vials, so no mixing or contamination shuold occur. Therefore, the QAS species will not be included in the pipette count for the experiment. 

We will assume that 2 small pipette tips and 2 large pipette tips will be used for their experiment. 

    NOTE:
    Even though we are using the p50 pipette, any volume between the maximim of the small pipette (p50) and the minimum of the large pipette (p1000), will be covered by the small pipette by successive dispensing. To avoid contamination of the stock after mixing of the samples, multiple pipettes will be used for a single transfer. Therefore, the "maximum" volume of the small pipette (50uL) will be changed to the "minimum" of the large one(100 uL) 

In [22]:
HBD_df_0 = run0.drop([QAS_dict['A0']+'-stock'], axis=1)
## the first 2 arguments correspond to the maximum volume
## a pipette can move expressed in uL
print('Run0 \n{} \n'.format(QAS_dict['A0']))

ALH.get_pipette_tip_count(100,1000, HBD_df_0)

Run0 
Choline Chloride 

 Small pipette tips: 14
 Small tipracks: 0.1
 Large pipette tips: 226
 Large tipracks: 2.35


In [25]:
HBD_df_1 = run1.drop([QAS_dict['A1']+'-stock',QAS_dict['A2']+'-stock'], axis=1)
print('Run1 \n{} & {} \n'.format(QAS_dict['A1'],QAS_dict['A2']))
ALH.get_pipette_tip_count(100,1000, HBD_df_1)

Run1 
Acetylcholine Chloride & Tetraethylammonium Chloride 

 Small pipette tips: 35
 Small tipracks: 0.4
 Large pipette tips: 445
 Large tipracks: 4.64


In [26]:
HBD_df_2 = run2.drop([QAS_dict['A3']+'-stock',QAS_dict['A4']+'-stock'], axis=1)
print('Run2 \n{} & {} \n'.format(QAS_dict['A3'],QAS_dict['A4']))
ALH.get_pipette_tip_count(100,1000, HBD_df_2)

Run2 
Tetrapropylammonium Bromide & Tetraethylammonium Iodide 

 Small pipette tips: 79
 Small tipracks: 0.8
 Large pipette tips: 401
 Large tipracks: 4.18


The following is the deck desing for each run

![HT_DES_OT2_Deck_Schematic.png](HT_DES_OT2_Deck_Schematic.png)

# Run0

#### Set up Loaded Dictionary and Directions for Volume Handling
It is important to know how many stock container will be needed for each stock and be able to provide the correct instructions for OT2 commands to follow.

The ranges of stock refers to the ranges of wells one stock will cover. The range is provided in terms of the index of the stock volume dataframe i.e.([lower, upper])). The format is as follows = [[stock1_range1, stock1_range2....], [stock2_range1, stock2_range2....] []...]. 

The range is based on the provided maximum volume for each container, since variabiltiy exist between filling it is **HIGHLY RECCOMENDED** to underestimate the total volume of stock a container can hold so for a 20mL scitillation vials, 16000-17000 uL should be use as the max. Another reason for this is the OT2 will do its best to pick up as much volume as possible but understand liquid in vials can coat the sides not leaving enough depth for the OT2 to grab, so it is **HIGHLY RECCOMENDED** to have at least a **15-20%** volume buffer for each stock

The basis of range is on the maximum volume of the stock container, however currently limitation are: 
 * All stock containers must be the same

In [27]:
labware_dir_path = r"./Custom Labware"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)
# custom_labware_dict

In [28]:
path = r"./DES_Campaign_Protocol.csv"
chem_path = r"Chemical Database.csv"
plan = CreateSamples.get_experiment_plan(path, chem_path)

In [31]:
QAS_0= run0[[QAS_dict['A0']+'-stock']]
HBD_0 = run0.drop([QAS_dict['A0']+'-stock'], axis=1)

In [32]:
protocol = simulate.get_protocol_api('2.8', extra_labware=custom_labware_dict)
loaded_dict = ALH.loading_labware(protocol, plan, well_order='column')
QAS_stock_info = ALH.stock_well_ranges(QAS_0, loaded_dict['Stock Wells'][:6], volume_buffer_pct=15)
HBD_stock_info = ALH.stock_well_ranges(HBD_0, loaded_dict['Stock Wells'][6:], volume_buffer_pct=15) 
stock_position_info = {**QAS_stock_info, **HBD_stock_info}
stock_position_info

C:\Users\pozzo_admin\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\pozzo_admin\.opentrons\deck_calibration.json not found. Loading defaults


{'Choline Chloride-stock': {'Ranges': [[0, 120]],
  'Total Volume': 36881.36951420089,
  'Stock Wells': [A1 of Agilent 6 Reservoir 47000 ÂµL on 1]},
 'Ethylene Glycol-stock': {'Ranges': [[0, 120]],
  'Total Volume': 2898.8401530018155,
  'Stock Wells': [A1 of USA Scientific 12 Well Reservoir 22 mL on 4]},
 'Glycerol-stock': {'Ranges': [[0, 120]],
  'Total Volume': 2898.8401530018155,
  'Stock Wells': [A2 of USA Scientific 12 Well Reservoir 22 mL on 4]},
 'Acetamide-stock': {'Ranges': [[0, 120]],
  'Total Volume': 2898.8401530018155,
  'Stock Wells': [A3 of USA Scientific 12 Well Reservoir 22 mL on 4]},
 'N,N-dimethylurea-stock': {'Ranges': [[0, 120]],
  'Total Volume': 2898.8401530018155,
  'Stock Wells': [A4 of USA Scientific 12 Well Reservoir 22 mL on 4]},
 '3-phenylpropionic acid-stock': {'Ranges': [[0, 120]],
  'Total Volume': 4096.246347083595,
  'Stock Wells': [A5 of USA Scientific 12 Well Reservoir 22 mL on 4]},
 'Urea-stock': {'Ranges': [[0, 120]],
  'Total Volume': 2898.840153

In [33]:
QAS_directions = ALH.create_sample_making_directions(QAS_0, stock_position_info, loaded_dict, start_position=0)
HBD_directions = ALH.create_sample_making_directions(HBD_0, stock_position_info, loaded_dict, start_position=0)


In [34]:
ALH.pipette_volumes_component_wise(protocol, QAS_directions, loaded_dict, mix_before=(1,500), new_tip='never', blow_out =True, blowout_location='destination well', touch_tip = True)

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 7
Transferring 97.67441860465117 from A1 of Agilent 6 Reservoir 47000 ÂµL on 1 to A1 of Nanosampler 48 Well Plate 1650 ÂµL on 2
Mixing 1 times with a volume of 50.0 ul
Aspirating 50.0 uL from A1 of Agilent 6 Reservoir 47000 ÂµL on 1 at 100.0 uL/sec
Dispensing 50.0 uL into A1 of Agilent 6 Reservoir 47000 ÂµL on 1 at 300.0 uL/sec
Aspirating 48.83720930232558 uL from A1 of Agilent 6 Reservoir 47000 ÂµL on 1 at 100.0 uL/sec
Touching tip
Dispensing 48.83720930232558 uL into A1 of Nanosampler 48 Well Plate 1650 ÂµL on 2 at 300.0 uL/sec
Touching tip
Blowing out at A1 of Nanosampler 48 Well Plate 1650 ÂµL on 2
Mixing 1 times with a volume of 50.0 ul
Aspirating 50.0 uL from A1 of Agilent 6 Reservoir 47000 ÂµL on 1 at 100.0 uL/sec
Dispensing 50.0 uL into A1 of Agilent 6 Reservoir 47000 ÂµL on 1 at 300.0 uL/sec
Aspirating 48.83720930232558 uL from A1 of Agilent 6 Reservoir 47000 ÂµL on 1 at 100.0 uL/sec
Touching tip
Dispensing 48.837209302

In [35]:
ALH.pipette_volumes_component_wise(protocol, HBD_directions, loaded_dict, mix_before=(1,300), mix_after=(2,300), new_tip='always', blow_out=True, blowout_location='destination well', touch_tip = True)

Out of bounds move: X=(419.2 motor controller, 419.2 deck) too high for limit 418.0
Out of bounds move: X=(419.2 motor controller, 419.2 deck) too high for limit 418.0
Out of bounds move: X=(419.2 motor controller, 419.2 deck) too high for limit 418.0
Out of bounds move: X=(419.2 motor controller, 419.2 deck) too high for limit 418.0
Out of bounds move: X=(419.2 motor controller, 419.2 deck) too high for limit 418.0
Out of bounds move: X=(419.2 motor controller, 419.2 deck) too high for limit 418.0
Out of bounds move: X=(419.2 motor controller, 419.2 deck) too high for limit 418.0
Out of bounds move: X=(419.2 motor controller, 419.2 deck) too high for limit 418.0
Out of bounds move: X=(419.2 motor controller, 419.2 deck) too high for limit 418.0
Out of bounds move: X=(419.2 motor controller, 419.2 deck) too high for limit 418.0
Out of bounds move: X=(419.2 motor controller, 419.2 deck) too high for limit 418.0
Out of bounds move: X=(419.2 motor controller, 419.2 deck) too high for limi

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 7
Transferring 97.67441860465117 from A1 of Agilent 6 Reservoir 47000 ÂµL on 1 to A1 of Nanosampler 48 Well Plate 1650 ÂµL on 2
Mixing 1 times with a volume of 50.0 ul
Aspirating 50.0 uL from A1 of Agilent 6 Reservoir 47000 ÂµL on 1 at 100.0 uL/sec
Dispensing 50.0 uL into A1 of Agilent 6 Reservoir 47000 ÂµL on 1 at 300.0 uL/sec
Aspirating 48.83720930232558 uL from A1 of Agilent 6 Reservoir 47000 ÂµL on 1 at 100.0 uL/sec
Touching tip
Dispensing 48.83720930232558 uL into A1 of Nanosampler 48 Well Plate 1650 ÂµL on 2 at 300.0 uL/sec
Touching tip
Blowing out at A1 of Nanosampler 48 Well Plate 1650 ÂµL on 2
Mixing 1 times with a volume of 50.0 ul
Aspirating 50.0 uL from A1 of Agilent 6 Reservoir 47000 ÂµL on 1 at 100.0 uL/sec
Dispensing 50.0 uL into A1 of Agilent 6 Reservoir 47000 ÂµL on 1 at 300.0 uL/sec
Aspirating 48.83720930232558 uL from A1 of Agilent 6 Reservoir 47000 ÂµL on 1 at 100.0 uL/sec
Touching tip
Dispensing 48.837209302

Mixing 2 times with a volume of 50.0 ul
Aspirating 50.0 uL from B5 of Nanosampler 48 Well Plate 1650 ÂµL on 3 at 150.0 uL/sec
Dispensing 50.0 uL into B5 of Nanosampler 48 Well Plate 1650 ÂµL on 3 at 400.0 uL/sec
Aspirating 50.0 uL from B5 of Nanosampler 48 Well Plate 1650 ÂµL on 3 at 150.0 uL/sec
Dispensing 50.0 uL into B5 of Nanosampler 48 Well Plate 1650 ÂµL on 3 at 400.0 uL/sec
Touching tip
Blowing out at B5 of Nanosampler 48 Well Plate 1650 ÂµL on 3
Dropping tip into A1 of Opentrons Fixed Trash on 12
Transferring 494.1176467970718 from A7 of USA Scientific 12 Well Reservoir 22 mL on 4 to C5 of Nanosampler 48 Well Plate 1650 ÂµL on 3
Picking up tip from A9 of Opentrons 96 Tip Rack 1000 µL on 10
Mixing 1 times with a volume of 50.0 ul
Aspirating 50.0 uL from A7 of USA Scientific 12 Well Reservoir 22 mL on 4 at 150.0 uL/sec
Dispensing 50.0 uL into A7 of USA Scientific 12 Well Reservoir 22 mL on 4 at 400.0 uL/sec
Aspirating 494.1176467970718 uL from A7 of USA Scientific 12 Well Reservo

### Step 10: Format for Exporting and Upload to Google Drive
* Using the information from our previously created dataframes we create a final dataframe to convert to a csv and upload to google drive for storage. 
    * Currently the two main pieces of information uploaded are composition of sample and sample location information. This for each sample is tied to a unique ID which contains a well_timestamp_keyword. 

* Currently the function *CreateSamples.add_final_location* uses direciton information plus the sampling dataframe either created or imported to add useful information and export as a csv.

In [28]:
filename = 'DES_run0'

In [38]:
QAS_fractions = [0.1,0.2,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.7,0.8,0.9]*10
HBD_fractions = [0.9,0.8,0.7,0.65,0.6,0.55,0.5,0.45,0.4,0.3,0.2,0.1]*10

In [39]:
final_directions = ALH.create_sample_making_directions(run0, stock_position_info, loaded_dict, start_position=0)
export_df = CreateSamples.add_final_location(final_directions,run0)
export_df['QAS_xf']= QAS_fractions
export_df['HBD_xf']= HBD_fractions
# export_df.to_csv('./'+filename+'.csv')
export_df

,UID,Labware,Slot,Well,Choline Chloride-stock,Ethylene Glycol-stock,Glycerol-stock,Acetamide-stock,"N,N-dimethylurea-stock",3-phenylpropionic acid-stock,Urea-stock,L-serine-stock,4-amino triazole-stock,Xylitol-stock,Phenylacetic acid-stock,Total Volume mL,QAS_xf,HBD_xf
0,S2_A1_01-26-2022,Nanosampler 48 Well Plate 1650 ÂµL,2,A1,97.674419,502.325581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,600.0,0.10,0.90
1,S2_B1_01-26-2022,Nanosampler 48 Well Plate 1650 ÂµL,2,B1,182.608696,417.391304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,600.0,0.20,0.80
2,S2_C1_01-26-2022,Nanosampler 48 Well Plate 1650 ÂµL,2,C1,257.142857,342.857143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,600.0,0.30,0.70
3,S2_D1_01-26-2022,Nanosampler 48 Well Plate 1650 ÂµL,2,D1,291.089109,308.910891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,600.0,0.35,0.65
4,S2_E1_01-26-2022,Nanosampler 48 Well Plate 1650 ÂµL,2,E1,323.076923,276.923077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,600.0,0.40,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,S5_B4_01-26-2022,Nanosampler 48 Well Plate 1650 ÂµL,5,B4,286.956522,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,313.043478,600.0,0.55,0.45
116,S5_C4_01-26-2022,Nanosampler 48 Well Plate 1650 ÂµL,5,C4,317.647059,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,282.352941,600.0,0.60,0.40
117,S5_D4_01-26-2022,Nanosampler 48 Well Plate 1650 ÂµL,5,D4,381.818182,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,218.181818,600.0,0.70,0.30
118,S5_E4_01-26-2022,Nanosampler 48 Well Plate 1650 ÂµL,5,E4,450.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150.000000,600.0,0.80,0.20


Repeat the same protocol for Run1 and Run2